In [2]:

import pytorch_lightning as pl
import torch
from torch import nn
from torchinfo import summary

from torchmetrics import functional as FM

In [3]:

class Model(pl.LightningModule): 
     
    def __init__(self):  ## 모델에 필요한 신경망 Layer 등 준비물은 init 에
        super().__init__()

        self.flatten = nn.Flatten()
        self.linear1 = nn.Linear(28*28, 32)
        self.linear2 = nn.Linear(28*28, 32)
        self.linear3 = nn.Linear(32+32, 10)
        self.relu = nn.ReLU()

    def forward(self, x): ## 그 신경망의 구성은 forward 에서 정의하는게 일반적임 물론, init 등에서 함수를 다 구성해두고 forward에서는 input-output 만 하게 해도 됨

        x = self.flatten(x)

        x1 = self.linear1(x)
        x1 = self.relu(x1) 
        
        x2 = self.linear2(x)
        x2 = self.relu(x2)

        x3 = torch.cat([x1, x2], dim=1)
        y = self.linear3(x3) 
        
        return(y)


In [17]:
model = Model()
summary(model, input_size = (1, 28, 28))

Layer (type:depth-idx)                   Output Shape              Param #
Model                                    [1, 10]                   --
├─Flatten: 1-1                           [1, 784]                  --
├─Linear: 1-2                            [1, 32]                   25,120
├─ReLU: 1-3                              [1, 32]                   --
├─Linear: 1-4                            [1, 32]                   25,120
├─ReLU: 1-5                              [1, 32]                   --
├─Linear: 1-6                            [1, 10]                   650
Total params: 50,890
Trainable params: 50,890
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 0.05
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.20
Estimated Total Size (MB): 0.21


----

하나의 모델이 다른 모델을 하위에 가지고 있는 형태,

한개의 모델에서 구성과 셋업을 다하고 바로 학습을 해도 된지만, 이렇게 할 경우 모델을 수정과 학습하는 Trainer 의 수정이 얽히게 된다.
간단한 구조일 때에는 별로 상관 없지만, 복잡한 경우를 대비해서 (그리고 여러가지 모델을 각각 테스트하는 등 많은 노가다가 필요할 경우)

모델은 여러개를 만들고

학습하는 애가 model1.forward 를 쓰다가 model2.forward 를 쓰는 식으로 간단히 바꿀 수 있도록 하는게 좋기 때문에 아래처럼

신경망의 구성을 하는 부분과, 학습하는 step부분을 정의한 클래스를 각각 두는게 유리하다.

In [4]:
loss_ftn = nn.CrossEntropyLoss()

class MyModel(pl.LightningModule):

    def __init__(self):
        super().__init__()
        self.layers = Model()  ## 1개의 모델을 통으로 가지고 있는 형태 (상속이랑은 미묘하게 다름 -> 여러개 모델을 동시에 거느리는걸 생각해보자)

    
    def forward(self, x):
        out = self.layers(x)
        return(out)


### 아래 step 부분은 우리가 직접 call 해서 쓸 수 도 있지만, 대부분(학습을 할 때 등)에는 Trainer 가 call 해서 사용한다.  따라서 거기에 맞게 
### arguments를 맞춰줘야 하기 때문에 사용하지 않는 batch_idx 등이 arg 에 포함됨.

    def predict_step(self, x, batch_idx):  ## pred 에서는 x 만 들어오기 때문에 batch 대신 x 라고 표시 
        y_pred = self(x) # 여기까진  logit 
        y_prob = nn.Softmax(y_pred)  # 확률로 변환 
        return(y_prob)

     
    def traning_step(self, batch, batch_idx): ## 학습시에는 (x, y) 쌍이 들어오므로 batch 라고 표현
        x, y = batch
        y_pred = self(x)
        loss = loss_fn(y_pred, y)

        acc = FM.accuracy(y_pred, y, task = 'multiclass', num_classes = 10) 
        mse = FM.mean_squared_error( torch.argmax( y_pred, dim=1 ), y)

        metrics = {'loss' : loss, 'acc' : acc, 'mse' : mse }
        self.log_dict( metrics, prog_bar = True) 

        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = loss_ftn(y_hat, y)    
        acc = FM.accuracy(y_hat, y, task = 'multiclass', num_classes = 10) 
        mse = FM.mean_squared_error( torch.argmax( y_hat, dim=1 ), y)
        metrics = {'val_loss' : loss, 'val_acc' : acc, 'val_mse' : mse }
        self.log_dict( metrics, prog_bar = True) 
        return 
        

    def test_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = loss_ftn(y_hat, y)    
        acc = FM.accuracy(y_hat, y, task = 'multiclass', num_classes = 10) 
        mse = FM.mean_squared_error( torch.argmax( y_hat, dim=1 ), y)
        metrics = {'test_loss' : loss, 'test_acc' : acc, 'test_mse' : mse }
        self.log_dict( metrics, prog_bar = True) 
        return 


    def configure_optimizers(self):
        return torch.optim.Adam( self.parameters(), lr=0.001 )




In [5]:
model = MyModel()
summary(model, input_size=(1, 28, 28))

Layer (type:depth-idx)                   Output Shape              Param #
MyModel                                  [1, 10]                   --
├─Model: 1-1                             [1, 10]                   --
│    └─Flatten: 2-1                      [1, 784]                  --
│    └─Linear: 2-2                       [1, 32]                   25,120
│    └─ReLU: 2-3                         [1, 32]                   --
│    └─Linear: 2-4                       [1, 32]                   25,120
│    └─ReLU: 2-5                         [1, 32]                   --
│    └─Linear: 2-6                       [1, 10]                   650
Total params: 50,890
Trainable params: 50,890
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 0.05
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.20
Estimated Total Size (MB): 0.21

모델을 만드는 법을 간단하게 복습해보았다. 이제, 다음장에서 이 모델을 학습하는 데이터 로드 모듈까지 ㄱㄱ